In [15]:
## TRAIN AND TUNE THRESHOLD FOR PRECISION

# Fit the pipeline
xgb_pipe.fit(X_train, y_train)

# Get predicted probabilities for the training set
train_probs = xgb_pipe.predict_proba(X_train)[:, 1]

# Evaluate thresholds
thresholds = np.linspace(0.4, 0.6, 50)  
best_threshold = None
best_precision = 0.0

for threshold in thresholds:
    train_predictions = (train_probs > threshold).astype(int)

    precision = precision_score(y_train, train_predictions)

    # Check if this threshold gives better precision
    if precision > best_precision:
        best_precision = precision
        best_threshold = threshold

# Print the best threshold found
print(f"Best threshold: {best_threshold} with precision score: {best_precision}")

## APPLY THE BEST THRESHOLD TO THE TEST SET

# Predict probabilities for the test set
test_probs = xgb_pipe.predict_proba(X_test)[:, 1]
test_predictions = (test_probs > best_threshold).astype(int)

# Evaluate precision on the test set with the tuned threshold
test_precision = precision_score(y_test, test_predictions)
print(f"Test set precision with tuned threshold: {test_precision}")

## PREDICT ON KAGGLE TEST SET AND SAVE SUBMISSION

# Predict probabilities for the Kaggle test set
kaggle_test_probs = xgb_pipe.predict_proba(X_df_kaggle_test_selected)[:, 1]

# Apply the best threshold to Kaggle test set predictions
kaggle_test_predictions = (kaggle_test_probs > best_threshold).astype(int)

# Prepare submission dataframe
df_kaggle_test = pd.read_csv("../data/test.csv")
df_kaggle_predicted_V5 = {
    "Patient ID": df_kaggle_test["Patient ID"],
    "Heart Attack Risk": kaggle_test_predictions,
}
df_kaggle_predicted_V5_xgb_precision = pd.DataFrame(df_kaggle_predicted_V5)

# Save submission to CSV
df_kaggle_predicted_V5_xgb_precision.to_csv(
    "../submission/df_kaggle_predicted_V5_xgb_precision.csv", index=False
)

# Number of cases in the unseen Kaggle test set
len(df_kaggle_test)


Best threshold: 0.5306122448979592 with precision score: 0.9994206257242179
Test set precision with tuned threshold: 0.3554987212276215


1753

In [16]:
sum(kaggle_test_predictions)

339